In [ ]:
import os
import time
import copy
import numpy as np
import torch

from torch import nn 
from torch import optim

from torchvision import datasets, models, transforms

In [ ]:
ddir = '/content/drive/MyDrive/study/study by lecture/implementation/dataset/hym_data'

batch_size = 24
num_workers = 2


data_transformers = {
    # compose는 여러개의 transform을 하나로 묶어주는 역할
    'train' : transforms.Compose(
        [
          transforms.RandomResizedCrop(224),
          transforms.RandomHorizontalFlip(),
          transforms.ToTensor(),
          transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230]) # 처음리스트가 RGB각각의 평균, 두번째 리스트는 RGB각각 표준편차
        ]
    ),
    'val' : transforms.Compose(
        [
          transforms.Resize(256),
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230])
        ]
    )
}
img_data = {
    k: datasets.ImageFolder(os.path.join(ddir,k),data_transformers[k])
    for k in ['train', 'val']
}
dloaders = {
    k: torch.utils.data.DataLoader(
        img_data[k], batch_size = batch_size, shuffle = True, num_workers=num_workers
    )
    for k in ['train', 'val']
}
dset_sizes = {x: len(img_data[x]) for x in ['train', 'val']}
classes = img_data['train'].classes

dvc = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
cfg = {
    'vgg16' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256,      'M', 512, 512, 512,      'M', 512, 512, 512,      'M'],
    'vgg19' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}

class VGG(nn.Module):
  def __init__(self, features, num_classes = 100):
    super().__init__()

    self.features = features

    self.classifier = nn.Sequential(
        nn.Linear(512*7*7, 4096),
        nn.ReLU(inplace = True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace = True),
        nn.Dropout(),
        nn.Linear(4096, num_classes)
    )

  def forward(self, x):
    output = self.features(x)
    output = output.view(output.size()[0], -1)
    output = self.classifier(output)
    return output

def make_layers(cfg):
  layers = []
  input_channel = 3

  for l in cfg:
    if l == 'M':
      layers += [nn.MaxPool2d(kernel_size = 2, stride = 2)]
      continue

    layers += [nn.Conv2d(input_channel, l, kernel_size= 3, padding = 1)]  
    layers += [nn.BatchNorm2d(l)]
    layers += [nn.ReLU(inplace=True)]

    input_channel = l
  return nn.Sequential(*layers)

def vgg16(num = 100):
  return VGG(make_layers(cfg['vgg16']), num_classes = num)
def vgg19(num = 100):
  return VGG(make_layers(cfg['vgg19']), num_classes = num)

In [ ]:
def train(model, loss_func, optimizer, epochs = 10):
  time_start = time.time()

  accuracy = 0.0

  for e in range(epochs):
    print(f'Epoch number {e}/{epochs - 1}')
    print('=' *20)

    for dset in ['train', 'val']:
      if dset == 'train':
        model.train()
      else:
        model.eval()
      
      loss = 0.0
      successes = 0
    
      for imgs, tgts in dloaders[dset]:
        imgs = imgs.to(dvc)
        tgts = tgts.to(dvc)
        optimizer.zero_grad()

        with torch.set_grad_enabled(dset == 'train'): # set_grad_enabled 찾아보기
          ops = model(imgs)
          _, preds = torch.max(ops, 1)
          loss_curr = loss_func(ops, tgts)

          if dset =='train':
            loss_curr.backward()
            optimizer.step()

        loss += loss_curr.item() * imgs.size(0)
        successes += torch.sum(preds == tgts.data)

      loss_epoch = loss / dset_sizes[dset]
      accuracy_epoch = successes.double() / dset_sizes[dset]
      
      print(f'{dset} loss in this epoch : {loss_epoch}, accuracy in this epoch: {accuracy_epoch}')
      if dset == 'val' and accuracy_epoch > accuracy:
        accuracy = accuracy_epoch
      
  time_delta = time.time() - time_start
  print(f'Training finished in {time_delta // 60}mins {time_delta % 60}secs')
  print(f'Best validation set accuracy: {accuracy}')

  return model

In [ ]:
model = vgg19(2)
if torch.cuda.is_available():
  model = model.cuda()
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
pretrained_model = train(model, loss_func, optimizer, epochs = 5)

Epoch number 0/4
